In [10]:
import copy
import requests
import numpy as np
import pandas as pd
import db_dtypes
import ftfy
import re
import six
import tiktoken
import pandas_gbq
from unidecode import unidecode
from langchain.chains.combine_documents.base import BaseCombineDocumentsChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import psycopg2
from pgvector.psycopg2 import register_vector
import json
import os
from google.oauth2 import service_account
from google.cloud import bigquery
#from asyncio.log import logger
from util import transform_sentence
from logger import get_logger
import psycopg2.extras as extras 
import hashlib
from langchain.docstore.document import Document
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
import textwrap
import openai
import warnings
warnings.filterwarnings("ignore")


AttributeError: module 'pyarrow' has no attribute '__version__'

In [ ]:
class LoadData:
    
    def run_select_statement(select_statement: str):
        credentials = service_account.Credentials.from_service_account_file('/Users/rodrigomoraes/Library/CloudStorage/GoogleDrive-rg.moraes@totvs.com.br/My Drive/TOTVS LABS/key_SA_GCP/labs-poc-09feb4e7688e.json')

        project_id = 'labs-poc'
        client = bigquery.Client(credentials= credentials,project=project_id)

        try:
            # Perform a query.
            query_job = client.query(select_statement)  # API request
            result = query_job.result()
        except Exception as e:
            print(f'Fetching resuls from database failed. {e}\nSelect statement: {select_statement}')
            raise
    
        return result

In [ ]:
class DatabaseService:

    def __init__(self):
        db_user='tecsupport'
        db_password='?Hi((<={}F{nI=jp'
        db_database='tecsupport'
        db_port='5432'
        db_host='34.123.172.21'
        self.connection_str = f"host='{db_host}' port='{db_port}' dbname='{db_database}' user='{db_user}' password='{db_password}'"

    def _get_database_connection(self):
        return psycopg2.connect(self.connection_str)

    def run_select_statement(self, select_statement: str, vars=None):
        #logger = get_logger(__name__)
        
        try:
            conn = self._get_database_connection()
            register_vector(conn)
        except Exception as e:
            print(f'Connecting to database failed. {e}')
            return []
        try:
            cursor = conn.cursor()
            cursor.execute(select_statement, vars=vars)
            fields = [field_md[0] for field_md in cursor.description]
            result = cursor.fetchall()
            result = [dict(zip(fields, row)) for row in result]
        except Exception as e:
            print(f'Fetching resuls from database failed. {e}\nSelect statement: {select_statement}')
            conn.rollback()
            result = []

        return result
    
    def run_dml_statement(self, df: str, table: str, vars=None): 
        logger = get_logger(__name__)
        
        try:
            conn = self._get_database_connection()
            register_vector(conn)
        except Exception as e:
            logger.error(f'Connecting to database failed. {e}')
            return []
        
        tuples = [tuple(x) for x in df.to_numpy()] 
  
        cols = ','.join(list(df.columns)) 
        # SQL query to execute 
        query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
        cursor = conn.cursor() 
        try: 
            extras.execute_values(cursor, query, tuples) 
            conn.commit() 
        except (Exception, psycopg2.DatabaseError) as error: 
            print("Error: %s" % error) 
            conn.rollback() 
            cursor.close() 
            return 1
        
        query = "SELECT COUNT(*) as cnt FROM %s;" % (table) 
        cursor.execute(query)
        num_records = cursor.fetchone()[0]

        print("Number of vector records in table: ", num_records,"\n")

        # Create an index on the data for faster retrieval

        #calculate the index parameters according to best practices
        # num_lists = num_records / 1000
        # if num_lists < 10:
        #     num_lists = 10
        # if num_records > 1000000:
        #     num_lists = math.sqrt(num_records)

        #use the cosine distance measure, which is what we'll later use for querying
        # cursor.execute(f'CREATE INDEX ON {table} USING ivfflat (sentence_embedding vector_cosine_ops) WITH (lists = {num_lists});')
        # cursor.execute(f'CREATE INDEX idx ON {table} USING hnsw (sentence_embedding vector_l2_ops);')
        # conn.commit()

        cursor.close()  

        return

In [ ]:
EMBEDDINGS_SELECT_COLUMNS = ['ticket_id', 'ticket_comment', 'ticket_sentence_hash', 'module', 'product',
                             'sentence_source', 'ticket_status', 'created_at', 'updated_at', 'score', 'first_comment']

class DocumentSearchService:

    def __init__(self):
        self.database_service = DatabaseService()
        self.threshold = 65

    def ticket_summarization(self, sentence: str, llm):
        """Returns ticket summarization by comment.

        :param sentence: Sentence to summarization.
        :param llm: LLMChain object.
        """

        text_splitter = CharacterTextSplitter()
        texts = text_splitter.split_text(sentence)
        docs = [Document(page_content=t) for t in texts[:4]]
        
        # prompt_template = """Write a concise topic based on the following passages, in Brazilian Portuguese, 
        #                      disregarding all personal names, date, attach and url of the following:

        # Perguntar ao prompt sobre requisicao e dúvida, fornecendo os comentarios, explicar qual a duvida/problema, 
        # com os demais itens (nomes, url...)
        # with a custom prompt
        prompt_template = """Write a concise summary, in chronological order, in Brazilian Portuguese, 
                            disregarding all personal names, date, attach and url, of the following:


        {text}


        CONCISE SUMMARY:"""
        PROMPT = PromptTemplate(template=prompt_template,
                                input_variables=["text"])

        ## with intermediate steps
        chain = load_summarize_chain(llm,
                                    chain_type="map_reduce",
                                    return_intermediate_steps=True,
                                    map_prompt=PROMPT,
                                    combine_prompt=PROMPT)

        output_summary = chain({"input_documents": docs}, return_only_outputs=True)
        wrapped_text = textwrap.fill(output_summary['output_text'],
                                    width=100,
                                    break_long_words=False,
                                    replace_whitespace=False)
    
        return wrapped_text

    def embeddings_search_on_database(self, query_vec: np.array, product: str, module: str,
                                      threshold: int, similarity: str, ticket_id: int, batch: bool):

        table_name = 'tickets_embeddings'
        
        if batch == False:
            select_statement = f'''SELECT * FROM
                                    (
                                        SELECT *, 1 - (sentence_embedding {similarity} %s) as score 
                                        FROM public.{table_name}
                                        WHERE (product ~ 'ˆ{product}.*' OR 
                                                module ~ '{module}')
                                            and ticket_id <> {ticket_id}
                                    ) as filtered_kb
                                WHERE score > {threshold/100};
                            '''
        else:
            select_statement = f'''SELECT * FROM
                                    (
                                        SELECT te.ticket_id, 1 - (sentence_embedding {similarity} %s) as score,
                                        (
                                            SELECT
                                                expected_id
                                            FROM
                                                public.tickets_similares
                                            WHERE ticket_id = {ticket_id}
                                        ) as expected_id 
                                        FROM public.{table_name} te
                                        WHERE te.ticket_id <> {ticket_id}
                                    ) as filtered_kb
                                WHERE score > {threshold/100};
                            '''
                                            
        result = self.database_service.run_select_statement(select_statement, (query_vec,))
        
        return pd.DataFrame(result)

    def find_tickets_for_query(self, query: str, product: str, module: str, k: int, similarity: str, ticket_id: int, batch: bool):
        
        if batch == False:
            # Searching tickets using similarity of OpenAPI embeddings
            query_vec =  OpenAIEmbeddings(
                openai_api_base="https://proxy.dta.totvs.ai/",
                openai_api_key="sk-axyZ_tPhqNPbbywhdhhhKQ",
                model="text-embedding-3-small"  
                ).embed_query(query)
            query_vec = np.array(query_vec)
        else:
            query_vec = query

        results = self.embeddings_search_on_database(query_vec, product, module, self.threshold, similarity, ticket_id, batch)

        if results.empty:
            return results
        
        # Getting only results with score higher than threshold
        results = results[results["score"] >= self.threshold / 100].copy()

        # Ordering results by score
        results.sort_values(by="score", ascending=False, inplace=True)

        # Keeping only the highest rank per ticket'
        results.drop_duplicates(subset=['ticket_id'], keep="first", inplace=True)
        results = results.head(k)

        return results
    

In [ ]:
from joblib import Parallel, delayed
import time

# Create similar tickets from the sent spreadsheet
def similarity_ticket():

    select_statement = ("""
                        SELECT
                            CAST(ticket_id AS INT64) AS ticket_id,
                            expected_id
                        FROM
                            `labs-poc.custom_data.tickets_similares`
                        """)
    
    df = LoadData.run_select_statement(select_statement).to_dataframe()

    # Grava o resultado dos dados coletados no BQ e faz insert do Dataframe diretamente no Banco Vetorizado
    DatabaseService().run_dml_statement(df, 'tickets_similares')

    return df

def embedding_sentence(result_df, i, llm, embedding):

    time.sleep(2)

    # Cria os embeddings para inserir os registros.
    #for i, row in result_df.iterrows():
    sentence = result_df.at[i,'ticket_comment']
    summary = DocumentSearchService().ticket_summarization(sentence, llm)

    # Cria embeddings para ticket_comment
    query_vec = embedding.embed_query(sentence)
    query_vec = np.array(query_vec)

    # Atualiza os valores da linha atual
    hash_concat = str(result_df.at[i,'ticket_id']) + sentence
    hash_id = hashlib.md5(hash_concat.encode('utf-8')).hexdigest()
    result_df.at[i,'ticket_sentence_hash'] =  hash_id
    result_df.at[i,'summary'] = summary
    result_df.at[i,'sentence_embedding'] = query_vec
    result_df.at[i,'sentence_source'] = 'ticket_comment'

    # Grava o resultado dos dados coletados no BQ e faz insert do Dataframe diretamente no Banco Vetorizado
    DatabaseService().run_dml_statement(result_df.iloc[[i]], 'tickets_embeddings')

    # Cria nova linha com dados do Assunto
    ultima_linha = result_df.loc[i]
    nova_linha = ultima_linha.copy()
    sentence = nova_linha['subject']
    hash_concat = str(nova_linha['ticket_id']) + sentence
    hash_id = hashlib.md5(hash_concat.encode('utf-8')).hexdigest()

    # Cria embeddings para a sentença
    query_vec = embedding.embed_query(sentence)
    query_vec = np.array(query_vec)

    # Atualiza os valores da linha atual
    nova_linha['ticket_sentence_hash'] =  hash_id
    nova_linha['summary'] = summary
    nova_linha['sentence_embedding'] = query_vec
    nova_linha['sentence_source'] = 'subject'
    result_df = result_df.append(nova_linha, ignore_index=True)

    # Grava o resultado dos dados coletados no BQ e faz insert do Dataframe diretamente no Banco Vetorizado
    DatabaseService().run_dml_statement(result_df.tail(1), 'tickets_embeddings')

    # Cria nova linha com dados do Resumo
    sentence = summary
    ultima_linha = result_df.loc[i]
    nova_linha = ultima_linha.copy()
    hash_concat = str(nova_linha['ticket_id']) + sentence
    hash_id = hashlib.md5(hash_concat.encode('utf-8')).hexdigest()

    # Cria embeddings para a sentença
    query_vec = embedding.embed_query(sentence)
    query_vec = np.array(query_vec)

    nova_linha['ticket_sentence_hash'] =  hash_id
    nova_linha['summary'] = summary
    nova_linha['sentence_embedding'] = query_vec
    nova_linha['sentence_source'] = 'summary'
    result_df = result_df.append(nova_linha, ignore_index=True)

    # Grava o resultado dos dados coletados no BQ e faz insert do Dataframe diretamente no Banco Vetorizado
    DatabaseService().run_dml_statement(result_df.tail(1), 'tickets_embeddings')

    # Cria nova linha com dados do primeiro comentario
    ultima_linha = result_df.loc[i]
    nova_linha = ultima_linha.copy()
    sentence = nova_linha['first_comment']
    hash_concat = str(nova_linha['ticket_id']) + sentence + summary
    hash_id = hashlib.md5(hash_concat.encode('utf-8')).hexdigest()

    # Cria embeddings para a sentença
    query_vec = embedding.embed_query(sentence)
    query_vec = np.array(query_vec)

    # Atualiza os valores da linha atual
    nova_linha['ticket_sentence_hash'] =  hash_id
    nova_linha['summary'] = summary
    nova_linha['sentence_embedding'] = query_vec
    nova_linha['sentence_source'] = 'first_comment'
    result_df = result_df.append(nova_linha, ignore_index=True)

    # Grava o resultado dos dados coletados no BQ e faz insert do Dataframe diretamente no Banco Vetorizado
    DatabaseService().run_dml_statement(result_df.tail(1), 'tickets_embeddings')

    return result_df

select_statement = ("""
                    WITH tickets_all AS (
                        SELECT
                            ticket_id,
                            STRING_AGG(ticket_comment, '\u2561') AS ticket_comment,
                            MAX(subject) AS subject,
                            '' AS summary,
                            '' ticket_sentence_hash,
                            MAX(module_name) AS module,
                            MAX(product_name) AS product,
                            '' AS sentence_source,
                            MAX(ticket_status) AS ticket_status,
                            '[1,2,3]' AS sentence_embedding,
                            MAX(created_at) AS created_at,
                            MAX(updated_at) AS updated_at,
                        FROM
                            `labs-poc`.custom_data.tickets tr
                        GROUP BY
                            ticket_id
                        ),

                        first_contact AS (
                        SELECT
                            ts.ticket_comment AS first_comment,
                            ts.ticket_id
                        FROM
                            `labs-poc`.custom_data.tickets ts
                        INNER JOIN
                            tickets_all tr
                        ON
                            ts.ticket_id = tr.ticket_id
                        QUALIFY ROW_NUMBER() OVER(PARTITION BY ts.ticket_id ORDER BY ts.comment_created_at) = 1
                        )

                        SELECT
                            ta.*,
                            tr.first_comment
                        FROM
                            tickets_all ta
                        INNER JOIN
                            first_contact tr
                            ON tr.ticket_id = ta.ticket_id

                    """)
# Faz a leitura dos daods no BQ
df = LoadData.run_select_statement(select_statement).to_dataframe()

dados = df

# Instancia llm e embedding
llm = OpenAI(
            openai_api_base="https://proxy.dta.totvs.ai/",
            openai_api_key="sk-axyZ_tPhqNPbbywhdhhhKQ",
            temperature=0,
            model="gpt-4o",
            )

embedding =  OpenAIEmbeddings(
            openai_api_base="https://proxy.dta.totvs.ai/",
            openai_api_key="sk-axyZ_tPhqNPbbywhdhhhKQ",
            model="text-embedding-3-small"  
            )

# Usa o joblib para paralelizar o processamento. 10 jobs por vez usando o backend threading.
Parallel(n_jobs=10, backend='threading')(delayed(embedding_sentence)(
         dados,i, llm, embedding)
         for i, row in dados.iterrows())

# Gera dados na tabela de tickets similares, estes dados vem da planilha enviada para busca de ticket similares
similarity_ticket()

In [ ]:
# ticket_id: 19432291
# <->: This pgvector query operator calculates the Euclidean (L2) distance between two vectors.
# Busca por summary
# Tickes com maior similaridade: 
            # 19369960 = ok
            # 19440602
            # 19249692 = ok
            # 19129354
            # 19141108 = ok
            # 18827337    
            # 18844460 = ok
            # 18980613 = ok
query = """Acessar espelho de ponto 2018 - mensagem-> não utiliza Cartão de Ponto
    """

product = 'Datasul'
module = ' '

documents_df = DocumentSearchService().find_tickets_for_query(query, product, module, k=20, similarity = '<->')
documents_df

In [ ]:
# ticket_id: 19432291
# <=>: This operator computes the cosine similarity between two vectors. Cosine similarity
# Busca por summary
# Tickes com maior similaridade: 
            # 19369960 = ok
            # 19440602 = ok top 1 (deve haver hit)
            # 19249692 = ok top 2
            # 19129354
            # 19141108 = ok 
            # 18827337    
            # 18844460 = ok
            # 18980613 = ok top 3
query = """Erro ao aplicar index CRM pacote 12.1.2311-2"""
ticket_id = 19432291
product = 'Planos - Linha Datasul'
module = ' '

documents_df = DocumentSearchService().find_tickets_for_query(query, product, module, k=20, similarity='<=>', ticket_id=ticket_id, batch=False)
documents_df

In [4]:
from joblib import Parallel, delayed
import time
from tqdm import tqdm

def find_similarity(df, product, module, sentence, index, pbar):

    time.sleep(2)

    ticket_id = df.at[index, 'ticket_id']
    query_vec = np.array(df.at[index, 'sentence_embedding'])

    # Find similar tickets using cosine similarity
    documents_df = DocumentSearchService().find_tickets_for_query(query_vec, product, module, k=20, similarity='<=>', ticket_id=ticket_id, batch=True)
    
    if not documents_df.empty:
        results_df = documents_df

        # Sort the DataFrame by score in descending order
        df_sorted = results_df.sort_values(by='score', ascending=False)

        # Seleciona os três principais ticket_id
        top_1 = df_sorted.index[0]
        top_2 = df_sorted.index[1] if len(df_sorted) > 1 else None
        top_3 = df_sorted.index[2] if len(df_sorted) > 2 else None

        # Additional columns corresponding only to found indexes
        results_df['Top 1'] = results_df.index == top_1
        results_df['Top 2'] = results_df.index == top_2 if top_2 else False
        results_df['Top 3'] = results_df.index == top_3 if top_3 else False

        # Add top ticket IDs to the DataFrame
        results_df['sentence'] = sentence
        results_df['target'] = ticket_id

        # Sort the final DataFrame by score
        df_sorted_final = results_df.sort_values(by='score', ascending=False)

        return df_sorted_final

    return pd.DataFrame()  # Return an empty DataFrame if no results are found

# Ticket IDs to compare in embeddings
ticket_ids = [19432291, 19528296, 15893702, 18858827]
product = 'Datasul'
module = ''
sentence = 'subject'

# Connecting to the database
conn = DatabaseService()._get_database_connection()
cur = conn.cursor()

# Execute the query to fetch the embeddings for each ticket_id
ticket_inputs = []
cur.execute(f"""
            select te.ticket_id, sentence_embedding 
            from tickets_embeddings te
            INNER JOIN tickets_similares ts
                ON ts.ticket_id = te.ticket_id
            where sentence_source = '{sentence}'
            """)
ticket_inputs = cur.fetchall()
#ticket_inputs.extend(ticket_input)

dados = pd.DataFrame(ticket_inputs, columns=['ticket_id', 'sentence_embedding'])

# Close the connection
cur.close()
conn.close()

# Initialize the progress bar
with tqdm(total=len(dados), desc="Processing") as pbar:
    # Define a function to wrap find_similarity and update the progress bar
    def process_row(index):
        result = find_similarity(dados, product, module, sentence, index, pbar)
        pbar.update(1)
        return result

    # Use Parallel to process each row in parallel
    results = Parallel(n_jobs=-1, backend='threading')(delayed(process_row)(i) for i in range(len(dados)))

# Concatenate the DataFrames
final_df = pd.concat(results, ignore_index=True)
group_df = pd.concat(results, ignore_index=True) 

# Function to check if the ticket_id is in the expected_id
def check_found(row):
    expected_ids = [int(x.strip()) for x in row['expected_id'].split(',') if x.strip().isdigit()]
    return row['ticket_id'] in expected_ids

# Function to check if the expected_id is in the ticket_id summarization found and not_found
def check_expected(row):
    expected_ids = [int(x.strip()) for x in row['expected_id'].split(',') if x.strip().isdigit()]
    found_count = sum(1 for eid in expected_ids if eid in all_ticket_ids)
    not_found_count = sum(1 for eid in expected_ids if eid not in all_ticket_ids)
    return found_count, not_found_count

# Apply function and create found and not found columns
final_df['found'] = final_df.apply(lambda row: 1 if check_found(row) else 0, axis=1)
final_df['not_found'] = final_df.apply(lambda row: 0 if check_found(row) else 1, axis=1)

# Export the final DataFrame to Excel
final_df.to_excel('final_results.xlsx', index=False)

# List of all ticket_id for reference
all_ticket_ids = set(group_df['ticket_id'])

# Apply function and create found and not found columns
group_df[['found', 'not_found']] = group_df.apply(lambda row: pd.Series(check_expected(row)), axis=1)

# Keep only the necessary columns
group_df = group_df[['expected_id', 'sentence', 'target', 'found', 'not_found']]

# Group by target and aggregate the results
df_grouped = group_df.groupby('target').agg({
    'expected_id': 'first',   # or ' '.join para concatenar
    'sentence': 'first',
    'found': 'first',
    'not_found': 'first'
}).reset_index()

# Add a column accuracy
df_grouped['accuracy'] = df_grouped.apply(
    lambda row: (row['found'] / (row['found'] + row['not_found'])) if (row['found'] + row['not_found']) > 0 else 0, 
    axis=1
)

# Export the final DataFrame to Excel
df_grouped.to_excel('final_results_grouped.xlsx', index=False)

# # Print the sorted DataFrame
# print(final_df)


NameError: name 'DatabaseService' is not defined

In [9]:
from similarity_finder import SimilarityFinder

# Create an instance of SearchService
find_similarity = SimilarityFinder()

# Call the accuracy_metrics method with the correct arguments
product = 'Datasul'
module = ''
sentence = 'subject'

# Ensure you pass exactly three arguments besides 'self'
df_grouped = find_similarity.process_data(product, module, sentence)


AttributeError: module 'pyarrow' has no attribute '__version__'

In [ ]:
#This will display a bar chart of scores for each ticket_id, allowing you to visualize 
#how similar each document is to the query embedding.

import matplotlib.pyplot as plt
import seaborn as sns

# Plot the scores with top positions
# plt.figure(figsize=(10, 6))
# plt.bar(df_sorted_final.index, df_sorted_final['score'], color='skyblue')
# plt.scatter(df_sorted_final[df_sorted_final['Top 1']].index, df_sorted_final[df_sorted_final['Top 1']]['score'], color='red', label='Top 1')
# plt.scatter(df_sorted_final[df_sorted_final['Top 2']].index, df_sorted_final[df_sorted_final['Top 2']]['score'], color='green', label='Top 2')
# plt.scatter(df_sorted_final[df_sorted_final['Top 3']].index, df_sorted_final[df_sorted_final['Top 3']]['score'], color='orange', label='Top 3')
# plt.xlabel('Ticket ID')
# plt.ylabel('Score')
# plt.title('Scores of Tickets with Top Positions')
# plt.legend()
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

# Prepare data for visualization
#ids, scores = zip(*results)

# Select Top X documents
top_x = 5
top_x_df = final_df.head(top_x)
top_x_df = top_x_df.sort_values(by='score', ascending=True)
print(top_x_df)

plt.figure(figsize=(10, 8))
sns.barplot(x='score', y='ticket_id', data=top_x_df, palette='viridis', orient='h')
plt.xlabel('score')
plt.ylabel('Ticket ID')
plt.title(f'Top {top_x} Documents by score')
plt.show()

# Plot the scores
# plt.figure(figsize=(10, 6))
# #sns.barplot(x=list(ids), y=list(scores), color='skyblue')
# sns.barplot(x='ticket_id', y='score', data=df_sorted_final, color='skyblue')
# plt.xlabel('Ticket ID')
# plt.ylabel('score')
# plt.title('scores of Embeddings')
# plt.show()